<a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 200px; display: inline" alt="Python"/></a>

<center>
       <h1>XLNET for multi-class text classification</h1>
       <h2>Wael Ben Hadj Yahia, Dorian Hervé<h2>
</center>
  

Sources :
<ul>
<li>https://simpletransformers.ai/docs/usage/</li>     <li>https://simpletransformers.ai/docs/tips-and-tricks/</li>
</ul>


In [1]:
import pandas as pd
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import warnings
warnings.filterwarnings("ignore")
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import torch
from gensim.parsing.preprocessing import remove_stopwords

print("Imported libraries.")


DATA_PATH = "data/defi-ia-insa-toulouse"
train_df = pd.read_json(DATA_PATH+"/train.json")
test_df = pd.read_json(DATA_PATH+"/test.json")
train_label = pd.read_csv(DATA_PATH+"/train_label.csv")
cat = pd.read_csv(DATA_PATH+"/categories_string.csv")

print("Data loaded.")
train_df.head()

Imported libraries.
Data loaded.


,Id,description,gender
0,0,She is also a Ronald D. Asmus Policy Entrepre...,F
1,1,He is a member of the AICPA and WICPA. Brent ...,M
2,2,Dr. Aster has held teaching and research posi...,M
4,3,He runs a boutique design studio attending cl...,M
5,4,"He focuses on cloud security, identity and ac...",M


In [2]:
test_df['description'] = test_df['description'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
train_df['description'] = train_df['description'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

print("Removed accents.")

Removed accents.


In [3]:
train_df = pd.merge(train_df, train_label, how='inner', on='Id')
train_df.drop(train_df.columns[[0,2]],axis=1,inplace=True)
train_df.columns = ["text", "labels"]

print("Formatted data.")
train_df.head()

Formatted data.


,text,labels
0,She is also a Ronald D. Asmus Policy Entrepre...,19
1,He is a member of the AICPA and WICPA. Brent ...,9
2,Dr. Aster has held teaching and research posi...,19
3,He runs a boutique design studio attending cl...,24
4,"He focuses on cloud security, identity and ac...",24


In [ ]:
cuda_available = torch.cuda.is_available()
print("A GPU will be used for this code : ", cuda_available)

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=3, max_seq_length=120, train_batch_size=128, use_multiprocessing=True, overwrite_output_dir=True, save_steps=-1) #, use_early_stopping=True, early_stopping_delta=0.01, early_stopping_metric= "mcc", early_stopping_metric_minimize=False, early_stopping_patience=5, evaluate_during_training_steps= 1000

# Create a Classification Model
model = ClassificationModel(
    'xlnet', 'xlnet-base-cased',
    num_labels=28,
    args=model_args,
    use_cuda=cuda_available
)

# Train the model
model.train_model(train_df)

print("Starting predictions")

# Make predictions with the model
predictions, raw_outputs = model.predict(list(test_df.description))

print("Writing output in csv file")

test_df["Category"] = predictions
baseline_file = test_df[["Id","Category"]]
baseline_file.to_csv("xlnet_3epochs.csv", index=False)

print("Successfully reached end of code.")

<br/>

As you can see, the simpletransformers library allows us to declare, train and use transformers based models (such as BERT or XLNET) with only a few lines of code.

This model scored more than 80% on the test data.

Unfortunately, we didn't have enough time to further explore this approach (testing other models, taking longer text sequences, etc).